In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from datetime import datetime

data = pd.read_csv("my_sensor_stream.csv")
mean_val = data["sensor_value"].mean()
std_val = data["sensor_value"].std()

z_scores = (data["sensor_value"] - mean_val) / std_val

threshold = 3

data["z_score"] = z_scores
data["z_anomaly"] = np.where(np.abs(z_scores) > threshold, 1, 0)
model = IsolationForest(random_state=11, contamination=0.05)

values = data[["sensor_value"]]

model.fit(values)

scores = model.decision_function(values)
pred = model.predict(values)

data["iso_score"] = scores
data["iso_anomaly"] = np.where(pred == -1, 1, 0)
log_rows = []

for i in range(len(data)):

    if data.loc[i, "z_anomaly"] == 1 or data.loc[i, "iso_anomaly"] == 1:

        log_rows.append({
            "timestamp": data.loc[i, "timestamp"],
            "sensor_value": data.loc[i, "sensor_value"],
            "z_score": data.loc[i, "z_score"],
            "isolation_score": data.loc[i, "iso_score"],
            "detected_on": str(datetime.now())
        })


anomaly_log = pd.DataFrame(log_rows)

anomaly_log.to_csv("anomaly_log.csv", index=False)

print("anomaly_log.csv created.")
print("total detected :", len(anomaly_log))


anomaly_log.csv created.
total detected : 50
